# Various Classification Models

In [46]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
import seaborn as sns
import matplotlib.pyplot as plt
import os
import glob

In [3]:
n_cpu = os.cpu_count()
print("Number of CPUs:", n_cpu)

Number of CPUs: 8


In [4]:
path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "./data/Combined_EMG_data*"))
data_list = []
X_list = []
y_list = []

for i in csv_files:
      
    df = pd.read_csv(i)
    data_list.append(df)

for j in data_list:
    X = j[['EMG Sensor 1', 'EMG Sensor 2', 'EMG Sensor 3', 'EMG Sensor 4']].to_numpy().reshape(-1,4)
    y = j['Mode'].to_numpy()
    X_list.append(X)
    y_list.append(y)
    
X = np.vstack(X_list)
y = np.hstack(y_list)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

## KNN

In [11]:
KNN = KNeighborsClassifier()
values = range(1,31)
parameter_grid = {'n_neighbors': values}

grid = GridSearchCV(estimator=KNN, param_grid=parameter_grid, cv=10)

grid.fit(X_train, y_train)

KNN = KNeighborsClassifier(n_neighbors=grid.best_params_['n_neighbors'])

KNN.fit(X_train, y_train)
y_pred = KNN.predict(X_test)
probs = KNN.predict_proba(X_test)
score = KNN.score(X_test, y_test)

# print(f"Predictions: \n{y_pred}")
# print(f"Probabilities: \n{probs}")
print(f"K: {grid.best_params_['n_neighbors']}")
print(f"Score: {score}")

K: 14
Score: 0.9144385026737968


In [12]:
none = np.array([0.26,0.07,0.1,0.32]).reshape(-1,4)
pred = KNN.predict(none)
print(pred)

index = np.array([0.9, 0.1, 0.4, 0.5]).reshape(-1,4)
pred = KNN.predict(index)
print(pred)

middle = np.array([1.6, 0.2, 1.5, 1.6]).reshape(-1,4)
pred = KNN.predict(middle)
print(pred)

fourth = np.array([1.9, 0.3, 0.6, 1.1]).reshape(-1,4)
pred = KNN.predict(fourth)
print(pred)

pinky = np.array([0.6, 0.2, 0.6, 1.1]).reshape(-1,4)
pred = KNN.predict(pinky)
print(pred)

[0]
[1]
[2]
[3]
[4]


## Random Forest

In [31]:
RFC = RandomForestClassifier()

values = range(1,31)

parameter_grid = {'n_estimators': values, 'max_depth': values}

grid = GridSearchCV(estimator=RFC, param_grid=parameter_grid, n_jobs=n_cpu-1, cv=5)

grid.fit(X_train, y_train)

RFC = RandomForestClassifier(n_estimators=grid.best_params_['n_estimators'], max_depth=grid.best_params_['max_depth'])

RFC.fit(X_train, y_train)

y_pred = RFC.predict(X_test)

print(f"Features importances: \n{RFC.feature_importances_}")
print(f"Confusion matrix: \n{confusion_matrix(y_test, y_pred)}") # predicted vertical, actual horizontal
print(f"Number of estimators: {grid.best_params_['n_estimators']}")
print(f"Max depth: {grid.best_params_['max_depth']}")
print(f"Score: {RFC.score(X_test, y_test)}")

Features importances: 
[0.39788865 0.04189598 0.17134004 0.38887534]
Confusion matrix: 
[[93  0  0  0  2]
 [ 0 23  0  0  0]
 [ 0  1 15  1  2]
 [ 0  1  7 18  0]
 [ 0  0  1  1 22]]
Number of estimators: 19
Max depth: 4
Score: 0.9144385026737968


In [32]:
none = np.array([0.26,0.07,0.1,0.32]).reshape(-1,4)
pred = RFC.predict(none)
print(pred)

index = np.array([0.9, 0.1, 0.4, 0.5]).reshape(-1,4)
pred = RFC.predict(index)
print(pred)

middle = np.array([1.6, 0.2, 1.5, 1.6]).reshape(-1,4)
pred = RFC.predict(middle)
print(pred)

fourth = np.array([1.9, 0.3, 0.6, 1.1]).reshape(-1,4)
pred = RFC.predict(fourth)
print(pred)

pinky = np.array([0.6, 0.2, 0.6, 1.1]).reshape(-1,4)
pred = RFC.predict(pinky)
print(pred)

[0]
[1]
[2]
[3]
[4]


## Multilayer Perceptron

In [59]:
# 3 layers: input layer, output layer, and hidden layer
# input layers receives input signal, output performs prediction and classifiation, hiden layers perform computations
# hidden_layer_sizes - determines number of hidden layers to use and neurons in each

MLP = MLPClassifier(random_state=1, max_iter=4000)

parameter_grid = {
    'hidden_layer_sizes': [(200,),(300,),(400,),(50,100,),(100,100,),(50,75,100,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
} 

grid = GridSearchCV(MLP, parameter_grid, n_jobs=n_cpu-1, cv=5) 
grid.fit(X_train, y_train) 

MLP = MLPClassifier(hidden_layer_sizes=grid.best_params_['hidden_layer_sizes'], activation=grid.best_params_['activation'], solver=grid.best_params_['solver'], random_state=1, max_iter=4000)
MLP.fit(X_train, y_train)

probs = MLP.predict_proba(X_test)
preds = MLP.predict(X_test)
score = MLP.score(X_test, y_test)

print(f"Hidden layer sizes: {grid.best_params_['hidden_layer_sizes']}")
print(f"Activation: {grid.best_params_['activation']}")
print(f"Solver: {grid.best_params_['solver']}")
print(f"Score: {score}")

Hidden layer sizes: (300,)
Activation: relu
Solver: adam
Score: 0.9411764705882353


In [73]:
y_true, y_pred = y_test , MLP.predict(X_test)

print('Results on the test set:')
print(classification_report(y_true, y_pred))

Results on the test set:
              precision    recall  f1-score   support

           0       1.00      0.99      0.99        95
           1       0.82      1.00      0.90        23
           2       0.88      0.74      0.80        19
           3       0.95      0.81      0.88        26
           4       0.89      1.00      0.94        24

    accuracy                           0.94       187
   macro avg       0.91      0.91      0.90       187
weighted avg       0.94      0.94      0.94       187



In [72]:
none = np.array([0.26,0.07,0.1,0.32]).reshape(-1,4)
pred = MLP.predict(none)
print(pred)

index = np.array([0.9, 0.1, 0.4, 0.5]).reshape(-1,4)
pred = MLP.predict(index)
print(pred)

middle = np.array([1.6, 0.2, 1.5, 1.6]).reshape(-1,4)
pred = MLP.predict(middle)
print(pred)

fourth = np.array([1.9, 0.3, 0.6, 1.1]).reshape(-1,4)
pred = MLP.predict(fourth)
print(pred)

pinky = np.array([0.6, 0.2, 0.6, 1.1]).reshape(-1,4)
pred = RFC.predict(pinky)
print(pred)

[0]
[1]
[2]
[3]
[4]


### True Test Values

In [71]:
none = np.array([0.18,0.07,0.27,0.26]).reshape(-1,4)
pred = KNN.predict(none)
print(pred)

index = np.array([1.07, 0.08, 0.49, 0.61]).reshape(-1,4)
pred = MLP.predict(index)
print(pred)

middle = np.array([2.02, 0.12, 0.76, 1.42]).reshape(-1,4)
pred = MLP.predict(middle)
print(pred)

fourth = np.array([1.06, 0.16, 0.61, 1.25]).reshape(-1,4)
pred = MLP.predict(fourth)
print(pred)

pinky = np.array([0.55, 0.1, 0.28, 0.88]).reshape(-1,4)
pred = MLP.predict(pinky)
print(pred)

[0]
[1]
[2]
[3]
[4]
